In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hate-speech-and-offensive-language-detection/train.csv


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import datetime
import os
from keras import models
from keras import layers

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
dataset=pd.read_csv("/kaggle/input/hate-speech-and-offensive-language-detection/train.csv")

In [4]:
dataset.head(15)

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ..."
7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ..."
9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria..."


In [5]:
dataset[["class","tweet"]].head(10)

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
6,1,"!!!!!!""@__BrighterDays: I can not just sit up ..."
7,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
8,1,""" &amp; you might not get ya bitch back &amp; ..."
9,1,""" @rhythmixx_ :hobbies include: fighting Maria..."


In [6]:
label=np.asarray(dataset["class"])

In [7]:
label

array([2, 1, 1, ..., 1, 1, 2])

In [8]:
dash_line="-".join("" for x in range(50))
print(f"Shape of Dataset {dataset.shape}")
print(dash_line)
print(f"Total Tweets {len(dataset['tweet'])} and Classes {dataset['class'].unique()}")
print(dash_line)

Shape of Dataset (24783, 6)
-------------------------------------------------
Total Tweets 24783 and Classes [2 1 0]
-------------------------------------------------


# Cleaning unrelated charactor form texts

In [9]:
corpus = []
for i in range(0, 24783):
    review = re.sub('[^a-zA-Z]', ' ', dataset['tweet'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [10]:
corpus[:10]

['rt mayasolov woman complain clean hous amp man alway take trash',
 'rt mleew boy dat cold tyga dwn bad cuffin dat hoe st place',
 'rt urkindofbrand dawg rt sbabi life ever fuck bitch start cri confus shit',
 'rt c g anderson viva base look like tranni',
 'rt shenikarobert shit hear might true might faker bitch told ya',
 'madison x shit blow claim faith somebodi still fuck hoe',
 'brighterday sit hate anoth bitch got much shit go',
 'selfiequeenbri caus tire big bitch come us skinni girl',
 'amp might get ya bitch back amp that',
 'rhythmixx hobbi includ fight mariam bitch']

# Cleaning & Converting text  into sequence 

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [12]:
max_word=1000
tokenizer=Tokenizer(num_words=max_word)

In [13]:
tokenizer.fit_on_texts(corpus)

In [14]:
sequences=tokenizer.texts_to_sequences(corpus)

In [15]:
sequences[:10]

[[2, 277, 613, 546, 217, 17, 31, 92, 56, 15],
 [2, 94, 76, 445, 32, 76, 3, 417, 405],
 [2, 654, 2, 939, 65, 98, 8, 1, 123, 245, 806, 13],
 [2, 221, 211, 709, 21, 6, 528],
 [2, 13, 369, 251, 435, 251, 1, 196, 41],
 [186, 13, 579, 614, 940, 414, 42, 8, 3],
 [332, 37, 173, 1, 14, 126, 13, 18],
 [81, 427, 108, 1, 47, 163, 580, 25],
 [17, 251, 10, 41, 1, 43, 17, 190],
 [198, 1]]

In [16]:
max_len=100
data = pad_sequences(sequences,maxlen=max_len)

In [17]:
data[:5]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   2,
        277, 613, 546, 217,  17,  31,  92,  56,  15],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [18]:
for i in range(5):
    print(f"Data {i}")
    print(f"Original Text:\n {corpus[i]} \n \n")
    print(f"Sequence:\n{sequences[i]} \n \n")
    print(f"Tensor:\n {data[i]} \n \n")

Data 0
Original Text:
 rt mayasolov woman complain clean hous amp man alway take trash 
 

Sequence:
[2, 277, 613, 546, 217, 17, 31, 92, 56, 15] 
 

Tensor:
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   2 277 613 546 217  17  31  92  56  15] 
 

Data 1
Original Text:
 rt mleew boy dat cold tyga dwn bad cuffin dat hoe st place 
 

Sequence:
[2, 94, 76, 445, 32, 76, 3, 417, 405] 
 

Tensor:
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

# Spliting data into test and training 

In [19]:
X_train=data[:len(data)//2]
y_train=label[:len(data)//2]
X_test=data[len(data)//2:]
y_test=label[len(data)//2:]

In [20]:
X_train

array([[  0,   0,   0, ...,  92,  56,  15],
       [  0,   0,   0, ...,   3, 417, 405],
       [  0,   0,   0, ..., 245, 806,  13],
       ...,
       [  0,   0,   0, ..., 623,   1,  93],
       [  0,   0,   0, ...,   1,  17,  93],
       [  0,   0,   0, ..., 107,   1,  93]], dtype=int32)

In [21]:
y_train

array([2, 1, 1, ..., 1, 1, 1])

# Model 

In [22]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [23]:
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [24]:
cm

array([[ 544,    8,    2],
       [9589,  126,   40],
       [2045,   17,   21]])